In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

In [1]:
tips = sns.load_dataset('tips')
tips.head(3)

<IPython.core.display.Javascript object>

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


# Merge
Merging is for doing complex column-wise combinations of dataframes in a SQL-like way. If you don't know SQL joins then check out this resource sql joins and comment below (and maybe I'll make a video).

Two merge we need two dataframes, let's make them below:

In [7]:
tips_bill = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()
tips_tip = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()

del tips_bill['tip']
del tips_tip['total_bill']

In [8]:
tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [9]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

In [10]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [11]:
# we can merge on the indexes
#Column-Index Merge:
#MERGE=SQL

pd.merge(tips_bill, tips_tip, 
         right_index=True, left_index=True)

<IPython.core.display.Javascript object>

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [12]:
tips_bill.reset_index()

,sex,smoker,total_bill
0,Male,Yes,1337.07
1,Male,No,1919.75
2,Female,Yes,593.27
3,Female,No,977.68


In [14]:
#we can reset indexes and then merge on the columns - perhaps the easiest way
#Index level
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index(),
    on=['sex', 'smoker'],
    
)

<IPython.core.display.Javascript object>

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [18]:
# it can actually infer the above - but be very careful with this
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index()
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [19]:
# it can merge on partial column and index
pd.merge(
    tips_bill.reset_index(), 
    tips_tip,
    left_on=['sex', 'smoker'],
    right_index=True
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [22]:
# we can do any SQL-like functionality
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='left'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,NaN
3,Female,No,977.68,NaN


In [23]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='inner'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00


In [24]:
# and if you add an indicator...
pd.merge(
    tips_bill.reset_index().tail(3), 
    tips_tip.reset_index().head(3),
    how='outer',
    indicator=True
)

,sex,smoker,total_bill,tip,_merge
0,Male,No,1919.75,302.00,both
1,Female,Yes,593.27,96.74,both
2,Female,No,977.68,NaN,left_only
3,Male,Yes,NaN,183.07,right_only


In [25]:
# it can handle columns with the same name
pd.merge(tips_bill, 
         tips_bill, 
         right_index=True, 
         left_index=True,
         suffixes=('_left', '_right')
)

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes              593.27            593.27
       No               977.68            977.68

# Contatenation
Concatenating is for combining more than two dataframes in either column-wise or row-wise. The problem with concatenate is that the combinations it allows you to do are rather simplistic. That's why we need merge.

Concatenate can take as many data frames as you want, but it requires that they are specifically constructed. All of the dataframes you pass in will need to have the same index. So no more using columns as an index.

In [21]:
# this adds the dataframes together row wise
#axis : {0/'index', 1/'columns'}, default 0
    #The axis to concatenate along.
    
#join : {'inner', 'outer'}, default 'outer'
    #How to handle indexes on other axis (or axes).
    
#ignore_index : bool, default False
    #If True, do not use the index values along the concatenation axis. The
    #resulting axis will be labeled 0, ..., n - 1. This is useful if you are
    #concatenating objects where the concatenation axis does not have
    #meaningful indexing information. Note the index values on the other
    #axes are still respected in the join.*/
    
pd.concat([tips_bill, tips_bill, tips_tip], sort=True)

<IPython.core.display.Javascript object>

tip  total_bill
sex    smoker                    
Male   Yes        NaN     1337.07
       No         NaN     1919.75
Female Yes        NaN      593.27
       No         NaN      977.68
Male   Yes        NaN     1337.07
       No         NaN     1919.75
Female Yes        NaN      593.27
       No         NaN      977.68
Male   Yes     183.07         NaN
       No      302.00         NaN
Female Yes      96.74         NaN
       No      149.77         NaN

In [28]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

In [29]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [30]:
# this does it column wise
pd.concat([tips_bill, tips_tip], axis=1)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [33]:
# this does it row wise
pd.concat([tips_bill, tips_tip], axis=0)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [34]:
# and finally this will add on the dataset where it's from
pd.concat([tips_bill, tips_tip], sort=False, keys=['num0', 'num1'])

total_bill     tip
     sex    smoker                    
num0 Male   Yes        1337.07     NaN
            No         1919.75     NaN
     Female Yes         593.27     NaN
            No          977.68     NaN
num1 Male   Yes            NaN  183.07
            No             NaN  302.00
     Female Yes            NaN   96.74
            No             NaN  149.77

In [35]:
# and finally this will add on the dataset where it's from
pd.concat([tips_bill, tips_tip], sort=True, keys=['num0', 'num1'])

tip  total_bill
     sex    smoker                    
num0 Male   Yes        NaN     1337.07
            No         NaN     1919.75
     Female Yes        NaN      593.27
            No         NaN      977.68
num1 Male   Yes     183.07         NaN
            No      302.00         NaN
     Female Yes      96.74         NaN
            No      149.77         NaN

# Pandas DataFrame append() Method

In [22]:
#Definition and Usage:
    
#The append() method appends a DataFrame-like object at the end of the current DataFrame.

#The append() method returns a new DataFrame object, no changes are done with the original DataFrame.


import pandas as pd

data1 = {
  "age": [16, 14, 10],
  "qualified": [True, True, True]
}
df1 = pd.DataFrame(data1)

data2 = {
  "age": [55, 40],
  "qualified": [True, False]
}
df2 = pd.DataFrame(data2)

newdf = df1.append(df2)

In [24]:
df1

,age,qualified
0,16,True
1,14,True
2,10,True


In [25]:
df2

,age,qualified
0,55,True
1,40,False


In [23]:
newdf

,age,qualified
0,16,True
1,14,True
2,10,True
0,55,True
1,40,False
